In [ ]:
import csv
import os
from io import BytesIO
import base64
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from google import genai
from google.generativeai import GenerativeModel
from PIL import Image

# Datos para el archivo CSV
data = [
    ["maquina", "horas produccion", "horas parada"],
    ["Engel", 115, 5],
    ["Arburg", 95, 25],
    ["Husky", 119, 1],
    ["Haitian", 104, 16]
]

# Nombre del archivo
filename = "inyectoras_plastico.csv"

# Crear el archivo CSV
with open(filename, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(data)

print(f"Archivo CSV generado: {filename}")

# Generar el gráfico con Plotly (como en tu código original)
# Leer el archivo CSV generado
df = pd.read_csv(filename)

# Crear un gráfico de barras agrupadas
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Añadir barras para horas de producción
fig.add_trace(
    go.Bar(
        x=df['maquina'],
        y=df['horas produccion'],
        name='Horas Producción',
        marker_color='green'
    ),
    secondary_y=False
)

# Añadir barras para horas de parada
fig.add_trace(
    go.Bar(
        x=df['maquina'],
        y=df['horas parada'],
        name='Horas Parada',
        marker_color='red'
    ),
    secondary_y=False
)

# Añadir línea para el porcentaje de eficiencia (horas producción / (horas producción + horas parada))
df['eficiencia'] = df['horas produccion'] / (df['horas produccion'] + df['horas parada']) * 100
fig.add_trace(
    go.Scatter(
        x=df['maquina'],
        y=df['eficiencia'],
        name='% Eficiencia',
        mode='lines+markers',
        marker=dict(color='blue', size=10),
        line=dict(width=3)
    ),
    secondary_y=True
)

# Actualizar diseño del gráfico
fig.update_layout(
    title_text='Análisis de Producción de Inyectoras de Plástico',
    barmode='group',
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="center",
        x=0.5
    )
)

# Configurar ejes Y
fig.update_yaxes(title_text="Horas", secondary_y=False)
fig.update_yaxes(title_text="% Eficiencia", secondary_y=True)

# Mostrar el gráfico
#fig.show()  # No lo mostramos directamente.

# Convertir el gráfico Plotly a imagen PNG en memoria
img_bytes = fig.to_image(format="png")  # Genera la imagen en formato PNG
img = Image.open(BytesIO(img_bytes))  # Abre la imagen PNG con PIL

# API Key (Asegúrate de tener la API Key configurada como variable de entorno)
GOOGLE_API_KEY = os.environ.get("GOOGLE_API_KEY")

if not GOOGLE_API_KEY:
    raise ValueError("La variable de entorno GOOGLE_API_KEY no está definida.")

#genai.configure(api_key=GOOGLE_API_KEY)  # comento y funciona pero con error en la repuesta ***
#model = GenerativeModel(model_name='gemini-pro-vision', api_key=GOOGLE_API_KEY)   # reemplazo no anda
# Modelo Gemini Vision
model = GenerativeModel('gemini-pro-vision')   

# Prepara el prompt
prompt_text = """
Analyze the provided chart data and provide a concise summary of the key trends and insights,
focusing on the performance of each machine.  Mention which machines are performing well and which need improvement
and suggest a few possible reasons for the differences. Include specific data points from the graph in your analysis.
Limit your response to 100 words.
"""

# Generar la respuesta con Gemini
try:
    response = model.generate_content([prompt_text, img], stream=False)
    print(response.text)  # Imprime el análisis generado por Gemini
except Exception as e:
    print(f"Error al generar la respuesta: {e}")

# Opcional: Guardar el gráfico como imagen HTML (como en tu código original)
fig.write_html("inyectoras_analisis.html")